In [1]:
import ee
import geemap
import os
import numpy as np
from matplotlib import pyplot as plt

In [2]:
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 5)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [3]:
manaus_shp = '/Users/morgansteckler/Downloads/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)

In [4]:
#Select geom of interest
fc = manaus

In [5]:
#Calculate NDRE and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)

#Create cloud mask
cloudBitMask = ee.Number(2).pow(10).int()
cirrusBitMask = ee.Number(2).pow(11).int()
def maskS2clouds(image):
    qa = image.select('QA60')
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
    return image.updateMask(mask)

In [76]:
#Load sentinel-2 data
s2 = (ee.ImageCollection('COPERNICUS/S2')#Select S2 1-C level data
      .select(['B5','B8', 'QA60']) #Select necessary bands to reduce memory usage
      .filterBounds(fc) #Narrow data spatially
      .filterDate(ee.Date('2018-01-01'), ee.Date('2019-12-31')) #Select years of interest
      .sort('CLOUDY_PIXEL_PERCENTAGE', False) #Put smallest %cloud image on top of stack
      .map(addNDRE)) #calculate NDRE and add as band

#Load copernicus land cover data
lc = (ee.Image("COPERNICUS/Landcover/100m/Proba-V/Global/2015") #select 2015
      .select('discrete_classification') #choose classification band
      .clip(fc)) #clip to geom

In [77]:
#Create a masked s2 collection
masked = s2.map(maskS2clouds)

In [78]:
clipped = masked.map(lambda image: image.clip(fc))

In [10]:
info = clipped.first().propertyNames().getInfo()
print(info)

['DATATAKE_IDENTIFIER', 'SPACECRAFT_NAME', 'FORMAT_CORRECTNESS_FLAG', 'system:id', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'SOLAR_IRRADIANCE_B12', 'system:version', 'SOLAR_IRRADIANCE_B10', 'SOLAR_IRRADIANCE_B11', 'GENERATION_TIME', 'SOLAR_IRRADIANCE_B8A', 'SENSOR_QUALITY_FLAG', 'CLOUD_COVERAGE_ASSESSMENT', 'system:time_end', 'system:time_start', 'DATASTRIP_ID', 'PROCESSING_BASELINE', 'SENSING_ORBIT_NUMBER', 'GEOMETRIC_QUALITY_FLAG', 'SENSING_ORBIT_DIRECTION', 'GRANULE_ID', 'REFLECTANCE_CONVERSION_CORRECTION', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8', 'DATATAKE_TYPE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9', 'RADIOMETRIC_QUALITY_FLAG', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4', 'MEAN_INCIDENCE_ZENITH_ANGLE_B1', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1', 'MEAN_INCIDENCE_ZENITH_ANGLE_B4', 'MEAN_INCIDENCE_ZENITH_ANGLE_B3', 'MEAN_INCIDENCE_ZENITH_ANGLE_B2', 'MEAN_INCIDENCE_ZENITH_ANGLE_B9', 'MEAN_INCIDENCE_ZENITH_ANGLE_B8', 'MEAN_SOLAR_ZENITH_ANGLE', 'MGRS_TILE'

In [14]:
info2 = lc.propertyNames().getInfo()
print(info2)

['forest_type_class_names', 'forest_type_class_palette', 'copyright', 'references', 'processing_level', 'system:id', 'source', 'platform', 'discrete_classification_class_values', 'institution', 'product_tags', 'contact', 'system:version', 'orbit_type', 'forest_type_class_values', 'discrete_classification_class_names', 'creator', 'product_version', 'system:time_end', 'NODATA_value', 'system:time_start', 'discrete_classification_class_palette', 'system:description', 'production_grid', 'processing_mode', 'sensor', 'system:asset_size', 'production_crs', 'doi', 'system:index', 'system:bands', 'system:band_names']


In [93]:
ndre_col = clipped.select('NDRE') #Just to make sure
ndre_img = ndre_col.first() #Grab one image to test
classed_img = ndre_img.addBands(lc)

In [82]:
Map.addLayer(classed_col, {}, 'test')

In [83]:
Map.addLayer(classed_img, {}, 'test img')

In [95]:
#Calculate mean per class
def classMean(img):
    reducer = (ee.Reducer
               .mean() #For some reason, mean wont work
               .group(groupField= 1, groupName= 'class'))
    return img.reduceRegion(reducer=reducer, geometry=fc, scale=20, maxPixels=1e8)

d = classMean(classed_img)
print(d.getInfo())

{'groups': [{'class': 20}, {'class': 30}, {'class': 40}, {'class': 112}, {'class': 116}, {'class': 122}, {'class': 126}]}
